# APRENDIZAJE SUPERVISADO

En este notebook se podrá visualizar el codigo relacionado con el aprendizaje supervisado unicamente. Para visualizar el codigo relacionado con el aprendizaje no supervisado acceder al archivo de este mismo repositorio, no_supervisado.ipynb

## Importación de las librerías 

A continuación se importan las librerias necesarias. Adicionalmente se configuran unas variables de entorno para obtener los mismos resultados para las diferentes ejecuciones. 

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras import regularizers, initializers
import random
from sklearn.model_selection import train_test_split
import seaborn as sns


# Establecer semilla estática
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

Método personalizado para la tasa de aprendizaje que se utiliza posteriormente

In [ ]:
# Custom learning rate schedule
def custom_lr_schedule(epoch):
    initial_lr = 1e-5
    peak_lr = 1e-2
    warmup_epochs = 10
    final_epoch = 200
    final_lr = 1e-3
    if epoch < warmup_epochs:
        lr = initial_lr + (peak_lr - initial_lr) * (epoch / warmup_epochs)
    else:
        decay_rate = np.log(final_lr / peak_lr) / (final_epoch - warmup_epochs)
        lr = peak_lr * np.exp(decay_rate * (epoch - warmup_epochs))
    return lr

Obtención de los datos

In [ ]:
# Load data
file_path = 'supervisado_final.csv'
df = pd.read_csv(file_path, sep=';', decimal='.')

# Data processing
X = df.drop(['ID Usuario', 'Conclusion'], axis=1).values
y = df['Conclusion'].values

Mapeo de etiquetas a enteros, codificación de etiquetas, escalado de datos y calculo de pesos de las calses

In [ ]:
# Map labels to integers
label_mapping = {0: 0, 0.5: 1, 1: 2}
y_mapped = np.array([label_mapping[val] for val in y])

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_mapped)

# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weights = dict(enumerate(class_weights))

Dividir el conjunto de datos en train (70%), test (20%) y validation (10%)

In [ ]:
# Split data into train, validation, and test sets
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=SEED)  # 10% of the original data

Creación de la arquitectura de la red neuronal

In [ ]:
# Model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_scaled.shape[1],), kernel_initializer=initializers.glorot_uniform(), kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_initializer=initializers.glorot_uniform(), kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)),
    BatchNormalization(),
    Dropout(0.3),    
    Dense(64, activation='relu', kernel_initializer=initializers.glorot_uniform(), kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_initializer=initializers.glorot_uniform(), kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-5)),
    BatchNormalization(),
    Dropout(0.3), 
    Dense(len(encoder.classes_), activation='softmax', kernel_initializer=initializers.glorot_uniform())
])

Compilar modelo

In [ ]:
# Compile model
optimizer = Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Callbacks

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(custom_lr_schedule)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')

Entrenamiento del modelo

In [ ]:
# Train model
history = model.fit(X_train, y_train, epochs=200, batch_size=16, validation_data=(X_val, y_val), class_weight=class_weights, callbacks=[lr_scheduler, model_checkpoint, early_stopping])

Visualizar el comportamiento de la perdida y la precisión de entrenamiento y validación para las diferentes épocas

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Precisión de entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()

plt.show()

Obtener el modelo con mejores resultados

In [ ]:
# Load the best model
best_model = tf.keras.models.load_model('best_model.keras')

Evaluar el mejor modelo obtenido con los datos de test. 

In [ ]:
# Evaluate the best model on the test set and print the accuracy
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy Red neuronal: {accuracy * 100:.2f}%")

# Predict and evaluate the best model on the test set
y_pred = best_model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

# Convert the encoder.classes_ values to string once
target_names = [str(class_name) for class_name in encoder.classes_]

# Print classification report without duplicates
print(classification_report(y_test, y_pred_classes, target_names=target_names))

Visualizar matriz de confusión para los datos en test

In [ ]:
# Plot confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.title('Matriz de Confusión')
plt.show()

Visualizar curva ROC y AUC

In [ ]:
# Plot ROC curve
plt.figure(figsize=(8, 6))
for i in range(len(encoder.classes_)):
    fpr, tpr, _ = roc_curve(y_test == i, y_pred[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Clase {target_names[i]} (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

Visualizar curva de precision-recall

In [ ]:
# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
for i in range(len(encoder.classes_)):
    precision, recall, _ = precision_recall_curve(y_test == i, y_pred[:, i])
    plt.plot(recall, precision, label=f'Clase {target_names[i]}')
plt.xlabel('Recall')
plt.ylabel('Precisión')
plt.title('Curva Precision-Recall')
plt.legend(loc="lower left")
plt.show()